# Everything of Thoughts (XoT) implementation in DataRobot

Authors: yifu.gu@datarobot.com, greig.bloom@datarobot.com, mitsuo.yamamoto@datarobot.com

## Summary

This accelerator introduces the implementation and evaluation of XoT (Everything of Thoughts) in DataRobot, which is the latest approach to make generative AI "think like humans." In the world of generative AI, various methods (called thought generation) are being researched to help AI acquire more human-like "thinking patterns." In particular, XoT aims to produce more accurate answers by teaching generative AI the "thinking process." There are two main methods to achieve XoT:

* Chain-of-Thought (CoT)[1]: A method of thinking by connecting multiple thoughts like a chain and reasoning through them.

* Retrieval Augmented Thought Tree (RATT)[2]: A method of thinking by expanding multiple possibilities like tree branches and retrieving relevant information from the external knowledge base.

This accelerator explains how to implement these methods. Specifically, it introduces how to set up and compare three types of LLM prompts: direct, Chain-of-Thought, and RATT. "Direct" referring to the well-known "you are a helpful assistant." The accelerator also explains how to conduct performance evaluations using sample datasets, comparing the accuracy and efficiency of each method, and analyze using multiple evaluation metrics.

## Prerequisites

This script uses Pulumi to set up the DataRobot environment. If Pulumi is not already installed, install the CLI by following the instructions [here](https://www.pulumi.com/docs/iac/download-install/). After installing for the first time, restart your terminal and run:

```bash
pulumi login --local  # omit --local to use Pulumi Cloud (requires separate account)
```


## Setup

This section imports necessary Python packages and sets up the environment. Configure the DataRobot client and define paths for input data.

In [18]:
import os
import time

import datarobot as dr
from dotenv import load_dotenv
from drops import get_prompt_count, get_trace_data
import pandas as pd
import pulumi
from pulumi import automation as auto
import pulumi_datarobot as datarobot
from tqdm import tqdm

load_dotenv(override=True)

# display all columns
pd.set_option("display.max_columns", None)

In [19]:
os.environ["PULUMI_CONFIG_PASSPHRASE"] = "dr"
client = dr.Client()

path_csv = "https://s3.us-east-1.amazonaws.com/datarobot_public_datasets/ai_accelerators/ragbench_test_demo.csv"
df_sample = pd.read_csv(path_csv)
prompt_count = df_sample.shape[0]

In [20]:
def stack_up(project_name: str, stack_name: str, program: callable) -> auto.Stack:
    # create (or select if one already exists) a stack that uses our inline program
    stack = auto.create_or_select_stack(
        stack_name=stack_name, project_name=project_name, program=program
    )

    stack.refresh(on_output=print)

    stack.up(on_output=print)
    return stack


def destroy_project(stack: auto.Stack):
    """Destroy pulumi project"""
    stack_name = stack.name
    stack.destroy(on_output=print)

    stack.workspace.remove_stack(stack_name)
    print(f"stack {stack_name} in project removed")

## Create GenAI components

This section creates the components needed to implement three different prompt styles:

1. Single stage question-answering ("You are a helpful assistant")
2. Chain of thought reasoning
3. Retrieval-augmented thought tree (RATT)

For each style, prepare system prompts, LLM configurations, and evaluation datasets.

Note that while RATT typically achieves more accurate responses through multiple iterations of questioning using LLM outputs, in this implementation you can adopt an approach that aims to obtain accurate responses in a single pass by carefully crafting the input prompts.

In [21]:
# Read prompt from *.md
with open("system_prompt_chain_of_thought.md", "r") as file:
    system_prompt_chain_of_thought = file.read()
with open("system_prompt_ratt.md", "r") as file:
    system_prompt_ratt = file.read()

In [22]:
PROJECT = "XOT Evaluations Demo"
LLM_ID = "azure-openai-gpt-4-o-mini"


def make_vdb_and_playground():
    # Usecase
    use_case = datarobot.UseCase(resource_name=PROJECT)

    # Eval Dataset
    dataset_eval = datarobot.DatasetFromFile(
        resource_name=f"{PROJECT} - Dataset Eval",
        file_path=path_csv,
        use_case_ids=[use_case.id],
    )

    # Playground
    playground = datarobot.Playground(
        resource_name=f"{PROJECT} - Playground", use_case_id=use_case.id
    )

    # LLM BPs
    llm_bp_direct = datarobot.LlmBlueprint(
        resource_name=f"Direct BP",
        llm_id=LLM_ID,
        playground_id=playground.id,
        llm_settings=datarobot.LlmBlueprintLlmSettingsArgs(
            max_completion_length=4096,
            system_prompt="You are a helpful assistant. Answer the question in a direct manner. Be straightforward.",
            temperature=0.1,
            top_p=0.95,
        ),
        prompt_type="ONE_TIME_PROMPT",
    )
    llm_bp_cot = datarobot.LlmBlueprint(
        resource_name=f"Chain-of-thought BP",
        llm_id=LLM_ID,
        playground_id=playground.id,
        llm_settings=datarobot.LlmBlueprintLlmSettingsArgs(
            max_completion_length=4096,
            system_prompt=system_prompt_chain_of_thought,
            temperature=0.1,
            top_p=0.95,
        ),
        prompt_type="ONE_TIME_PROMPT",
    )
    llm_bp_ratt = datarobot.LlmBlueprint(
        resource_name=f"RATT BP",
        llm_id=LLM_ID,
        playground_id=playground.id,
        llm_settings=datarobot.LlmBlueprintLlmSettingsArgs(
            max_completion_length=4096,
            system_prompt=system_prompt_ratt,
            temperature=0.1,
            top_p=0.95,
        ),
        prompt_type="ONE_TIME_PROMPT",
    )
    pulumi.export("use_case_id", use_case.id)
    pulumi.export("dataset_eval_id", dataset_eval.id)
    pulumi.export("playground_id", playground.id)
    pulumi.export("llm_bp_direct_id", llm_bp_direct.id)
    pulumi.export("llm_bp_cot_id", llm_bp_cot.id)
    pulumi.export("llm_bp_ratt_id", llm_bp_ratt.id)

In [ ]:
project = "Evaluating_XoT_strategies"
stack_name = "eval_XoT_strategies_demo"
stack = stack_up(project_name=project, stack_name=stack_name, program=make_vdb_and_playground)

In [24]:
output = stack.outputs()
use_case_id = output["use_case_id"].value
playground_id = output["playground_id"].value
dataset_eval_id = output["dataset_eval_id"].value
playground_id = output["playground_id"].value
llm_bp_direct_id = output["llm_bp_direct_id"].value
llm_bp_cot_id = output["llm_bp_cot_id"].value
llm_bp_ratt_id = output["llm_bp_ratt_id"].value

## Running evaluations

This section uses the created components to perform actual evaluations.
The evaluation process consists of the following steps:

1. Set up evaluation datasets.
2. Define evaluation metrics (correctness, latency, token count, etc.).
3. Generate responses using each prompt style,
4. Collect and analyze results.



In [25]:
client = dr.Client()

In [26]:
eval_dataset_config = (
    dr.models.genai.evaluation_dataset_configuration.EvaluationDatasetConfiguration.create(
        name="XoT Evaluation Dataset",
        use_case_id=use_case_id,
        dataset_id=dataset_eval_id,
        prompt_column_name="text",
        playground_id=playground_id,
        is_synthetic_dataset=False,
        response_column_name="response",
    )
)

In [27]:
# ootb metrics
ootb_metrics = dr.models.genai.ootb_metric_configuration.PlaygroundOOTBMetricConfiguration.create(
    playground_id=playground_id,
    ootb_metric_configurations=[
        dr.models.genai.insights_configuration.InsightsConfiguration(
            **{
                "ootb_metric_name": "citations",
                "insight_name": "Citations",
            }
        ),
        dr.models.genai.insights_configuration.InsightsConfiguration(
            **{
                "ootb_metric_name": "latency",
                "insight_name": "Latency",
            }
        ),
        dr.models.genai.insights_configuration.InsightsConfiguration(
            **{
                "ootb_metric_name": "response_tokens",
                "insight_name": "Response tokens",
            }
        ),
        dr.models.genai.insights_configuration.InsightsConfiguration(
            **{
                "ootb_metric_name": "correctness",
                "insight_name": "Correctness",
                "llm_id": "azure-openai-gpt-4-o",
            }
        ),
    ],
)

In [28]:
insights_configuration = dr.models.genai.metric_insights.MetricInsights.list(
    playground=playground_id
)
path = "genai/evaluationDatasetMetricAggregations/"
payload = {
    "chat_name": "eval1",
    "llm_blueprint_ids": [llm_bp_direct_id, llm_bp_cot_id, llm_bp_ratt_id],
    "evaluation_dataset_configuration_id": eval_dataset_config.id,
    "insights_configuration": [insight.to_dict() for insight in insights_configuration],
}
response = client.post(path, json=payload)
job_id = response.json()["jobId"]

Retrieve the result from DataRobot.

Run the following only **AFTER** aggreration jobs on DataRobot finished.

In [ ]:
# wait for aggregation job to finish(=all prompts are processed)
while True:
    processed_count = get_prompt_count(
        playground_id=playground_id, headers=client.headers, endpoint=client.endpoint
    )
    print(f"Processed count: {processed_count}")
    if processed_count >= prompt_count * 3:  # incase other prompts are processed
        break
    time.sleep(30)

In [33]:
df_trace = get_trace_data(
    playground_id=playground_id, headers=client.headers, endpoint=client.endpoint
)

In [ ]:
df_trace.tail()

Use the following cells to generate a comparison chat.

In [ ]:
compare_chat = dr.models.genai.comparison_chat.ComparisonChat.create(
    "EvalCompare", playground=playground_id
)
compare_chat_id = compare_chat.id

In [ ]:
compare_prompt_ids = []

for q in tqdm(df_sample["text"]):
    compare_prompt = dr.models.genai.comparison_prompt.ComparisonPrompt.create(
        llm_blueprints=[llm_bp_direct_id, llm_bp_cot_id, llm_bp_ratt_id],
        text=q,
        comparison_chat=compare_chat_id,
        wait_for_completion=True,  # need to wait to submit the next prompt
    )
    compare_prompt_ids.append(compare_prompt.id)

In [ ]:
# create a link to open DataRobot
url = f"https://app.datarobot.com/usecases/{use_case_id}/playgrounds/{playground_id}/comparison/aggregations?comparisonChatId={compare_chat_id}&llmBlueprintId1={llm_bp_direct_id}&llmBlueprintId2={llm_bp_cot_id}&llmBlueprintId3={llm_bp_ratt_id}"
print(f"Comparison URL: {url}")

## Analyze results

This section analyzes the evaluation results and compares the performance of each prompt style (the result can be viewed in the DataRobot UI).


In [35]:
df_trace.groupby("llmBlueprintName")[
    [
        "metrics_correctness",
        "metrics_latency",
        "metrics_response_tokens",
    ]
].mean().reset_index()

,llmBlueprintName,metrics_correctness,metrics_latency,metrics_response_tokens
0,Chain-of-thought BP-587bec2,3.75,4.72108,384.38
1,Direct BP-3f2ec8d,3.22,0.81980,27.56
2,RATT BP-c04cd5e,3.07,8.49068,636.24


The results show that the Chain of Thought approach demonstrates higher accuracy compared to other approaches.
This suggests the effectiveness of explicitly making generative AI show its thought process.

## Clean up

In [ ]:
# # clean up
# project = "Evaluating_XoT_strategies"
# stack_name = "eval_XoT_strategies_demo"
# stack = auto.create_or_select_stack(
#     stack_name=stack_name, project_name=project, program=lambda: None
# )
# destroy_project(stack)